In [1]:
!pip3 install -q convertdate

You should consider upgrading via the '/usr/local/bin/python3.6 -m pip install --upgrade pip' command.


In [2]:
!pip3 install -q tqdm fbprophet  

You should consider upgrading via the '/usr/local/bin/python3.6 -m pip install --upgrade pip' command.


In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
from tqdm.notebook import tqdm 

 
plt.style.use('default')

## Data Ingest from Hive Table

In [4]:
from pyspark.sql import SparkSession
from pyspark_llap import HiveWarehouseSession


from pyspark.sql.types import *

spark = SparkSession\
    .builder\
    .appName("PythonSQL-Client")\
    .master("local[*]")\
    .config("spark.sql.hive.hiveserver2.jdbc.url", "jdbc:hive2://george-blog3-6.george-blog3.root.hwx.site:10000/default;principal=hive/_HOST@ROOT.HWX.SITE")\
    .config("spark.datasource.hive.warehouse.read.via.llap", "false")\
    .config("spark.datasource.hive.warehouse.read.jdbc.mode", "client")\
    .config("spark.datasource.hive.warehouse.metastoreUri", "thrift://george-blog3-6.george-blog3.root.hwx.site:9083")\
    .getOrCreate()


hive = HiveWarehouseSession.session(spark).build()
#sdf=hive.sql("select * from bdp.covid_data")
sdf=hive.sql("select iso_code, continent, rec_location, rec_date, people_fully_vaccinated from bdp.covid_data where people_fully_vaccinated > 0")
df = sdf.toPandas()

## Initial Data Preprocessing

- Convert updated datetime to datetime format
- Create new column of location_zip code (there are records with same location, different zip codes and different confirmed cases)
- Sort data for easy visualization
- Fill missing confirmed case data with zeros (13 cases)



In [ ]:
df.head(10)

In [ ]:
locations = list(df.rec_location.unique())
print("# unique locations", len(locations))

In [32]:
def plot_case_by_location(locations, df, title):
  num_cols =4
  fig, axes = plt.subplots(math.ceil(len(locations)/num_cols),num_cols  ) 
  fig.suptitle(title, fontsize=25);  
  for i in range(len(locations)):
    xdf = df[df.rec_location == locations[i]]
    xdf.plot(ax=axes[ int(i/num_cols), i%num_cols], x="rec_date", y=["people_fully_vaccinated"], title=locations[i]); 

  fig.set_figheight(len(locations))
  fig.set_figwidth(20)
  plt.tight_layout();
  fig.subplots_adjust(top=0.94)
  

In [ ]:
plot_case_by_location(locations[10:50], df, "News Vaccinated Users") 